# Classic timeseries models

The purpose of the naive models are to form a benchmark for performance. This allows us to compare the performance of feature engineering, hyperparameter tuning, and model architecture against a set of references.


### Test Harness: Walk forward validation
A test rig is a naive implementation of the prediction model. In this notebook univariate and multivariate rigs are deployed as benchmarks for improvement in predictions.

Naive univariate models are deployed using only the most recent data point in time to predict the next. I.e. t-1 = t. 

Multivariate models are deployed using all available feature vectors as predictors.  


#### Classic Models
- Univariate Naive
- Multivariate Naive
- Multivariate Moving Averages
- Multivariate ARIMA

###

In [2]:
#import required packages
import numpy as np
import pandas as pd
from tensorflow.keras import metrics

In [6]:
#load the preprocessed data
data = pd.read_csv('./data/processed/transformed_2016_2018.csv')

#investigate features for h_0
#y-hat is columns from t-0
data[['t-0 h_0','t-1 h_0']].head()

,t-0 h_0,t-1 h_0
0,25833.0,27662.0
1,22113.0,25833.0
2,26810.0,22113.0
3,27578.0,26810.0
4,26897.0,27578.0


In [49]:
def set_X_Y_features(data, y_target='t-0', univariate=False):
    """
    Function that takes in the preprocessed data and returns the Y and X datasets for univariate and multivariate test harnesses.
    
    """
    
    #if univariate is called, the function returns the current time step, and the previous time step.
    if univariate:
        y_target='t-0'
        #select the column headers to build Y matrix
        Y_cols = [col for col in data.columns if y_target == col.split()[0]]
        
        x_targets = 't-1'
        #selects the columns headers to build feature vectors of X
        X_cols = [col for col in data.columns if x_targets == col.split()[0]]
    
    #if not univariate then X and Y return the multivariate case with all features
    else:
        Y_cols = [col for col in data.columns if y_target == col.split()[0]]
        
        X_cols = [col for col in data.columns if y_target == col.split()[0]]
        
    #convert dataframe into numpy array
    Y = np.array(data[Y_cols])
    
    X = np.array(data[X_cols])
    
    return X, Y

In [50]:
X_univar, Y_univar = set_X_Y_features(data, univariate=True)
X_univar.shape, Y_univar.shape

((1066, 24), (1066, 24))

#### Split test and train sets

In [51]:
def split_train_test(X, Y, size):
    """
    Function to split data into train and test sets.
    
    """
    
    train_size = int(len(X) * size)
    
    X_train, X_test = X[:train_size], X[train_size:]
    
    Y_train, Y_test = Y[:train_size], Y[train_size:]
    
    return X_train, X_test, Y_train, Y_test

In [52]:
X_train_univar, X_test_univar, Y_train_univar, Y_test_univar = split_train_test(X_univar, Y_univar, 0.70)

print('X_train shape: {}'.format(X_train_univar.shape))
print('Y_train shape: {}'.format(Y_train_univar.shape))
print('X_test shape: {}'.format(X_test_univar.shape))
print('Y_test shape: {}'.format(Y_test_univar.shape))

X_train shape: (746, 24)
Y_train shape: (746, 24)
X_test shape: (320, 24)
Y_test shape: (320, 24)


### Evaluation metrics

In multi-output problems errors are typically evaluated indivually per output period as opposed to aggregating. Aggregating can still give an indication of the general model performace. Using the indivdual errors however is useful to identify the time steps we are predicting well, versus those that are not.


The base units of the problem are in MWh and having an error metric that is in these same units lets us make a direct comparison. Both Root Mean Squared Error and Mean Absolute Error are suitable for this task.

In [68]:
def calculate_errors(Y_hat_test, Y_test):
    
    columns = ['RMSE', 'MAE']
    
    error_list = []
    
    
    #calculate the mse and mae for each hour in the Y_test and Prediction
    for i in range(Y_hat_train.shape[1]):
        error_list.append([
            #calcualte the RMSE
            np.sqrt(metrics.mean_squared_error(Y_hat_test[:,i], Y_test[:,i]).numpy()),
            #calcualte the MAE
            metrics.mean_absolute_error(Y_hat_test[:,i], Y_test[:,i]).numpy()
        ])

        
    
    errors = pd.DataFrame(error_list, columns=columns)
    
    return errors
    

### Univariate Naive

The univariate naive forecast uses the previous time step as the prediction for the next timestep. As this is a multi-step problem, the naive univariate uses all h0..h23 slices from t-1 as predictions for t.

To calcualte the mean squared and absolute errors for this we will calculate the errors for each of h0...h23 for all time slices in the seires then sum these values. This gives an error for the full 24 hour ahead prediction.

In [53]:
def naive_univariate(X_train, X_test):
    return X_train, X_test
    

In [69]:
Y_hat_train, Y_hat_test = naive_univariate(X_train_univar, X_test_univar)

print(calculate_errors(Y_hat_train, Y_train_univar))
#print(calculate_errors(Y_hat_test, Y_test_univar))

             MSE          MAE
0   3.299129e+06  1305.699732
1   2.786931e+06  1199.639410
2   2.594002e+06  1157.549598
3   2.534801e+06  1171.447721
4   2.590720e+06  1205.084450
5   3.106111e+06  1351.450402
6   7.103553e+06  2026.420912
7   1.883022e+07  3195.315013
8   2.701522e+07  3798.644772
9   2.436017e+07  3633.500000
10  1.990717e+07  3303.415550
11  1.772679e+07  3126.454424
12  1.784318e+07  3145.319035
13  1.673891e+07  3041.687668
14  1.377310e+07  2740.873995
15  1.564674e+07  2915.660858
16  1.763742e+07  3101.571046
17  1.856147e+07  3187.349866
18  1.691876e+07  3034.004021
19  1.507685e+07  2834.851206
20  1.185563e+07  2501.213137
21  8.241524e+06  2079.643432
22  5.211170e+06  1603.162198
23  3.572319e+06  1311.462466


In [62]:
Y_hat_train.shape

(746, 24)

In [ ]:
for i in range(len(Y_hat_train.shape[1])):
    